# Products Export Job (SDK-Based)

This notebook runs a products export job using the SaaStify Edge SDK.

## Parameters (Papermill)

This notebook accepts the following parameters (matching productsExport.ipynb pattern):
- `job_id`: Request/job identifier (request_id in saas_edge_jobs table)
- `saas_edge_id`: Tenant identifier

The notebook will fetch job details from `saas_edge_jobs` table and extract:
- `template_id` from `request_args.template_id`
- `filter_params` from `request_args.filter_params`
- `job_path` from `request_args.job_path`
- `is_xlsx`, `is_json` from `request_args`
- `email` from `request_args.email`
- `export_with_readiness` from `request_args.export_with_readiness`

## Environment Variables Required

Set these before running:
- `DB_MODE`: Database connection mode (direct/proxy/local)
- `DB_INSTANCE`: Cloud SQL instance (for direct mode)
- `DB_NAME`: Database name
- `DB_USER`: Database user
- `DB_PASSWORD`: Database password
- `GOOGLE_APPLICATION_CREDENTIALS`: Path to GCS service account JSON (optional)


In [ ]:
# Papermill parameters - these will be injected when running with papermill
# Parameters match productsExport.ipynb pattern
job_id = "your-job-id-here"  # request_id in saas_edge_jobs table
saas_edge_id = "your-saas-edge-id-here"


## 1. Install Dependencies and Import SDK


In [ ]:
# Install dependencies (matching productsExport.ipynb)
%pip install saastify-edge-sdk pg8000 psycopg2-binary pandas requests openpyxl google-cloud-storage --quiet

# Import required modules
import asyncio
import json
import os
import traceback
import tempfile
from datetime import datetime
from typing import Dict, Any, List, Optional
from uuid import UUID
from io import BytesIO

# Import SDK components
from saastify_edge import ExportPipelineConfig, run_product_export
from saastify_edge.db import PostgreSQLClient, get_db_config, JobStatusUpdater
from saastify_edge.utils import setup_logging, set_job_context, get_or_generate_template_schema

# Import GCS upload helper
from google.cloud import storage
import pandas as pd
from openpyxl import Workbook

print("✅ Dependencies installed and SDK imported successfully")


## 2. Fetch Job Details from Database


In [ ]:
# Initialize database client to fetch job details
db_config = get_db_config()
db_client = PostgreSQLClient(db_config)
await db_client.connect()

# Fetch job details from saas_edge_jobs table (matching productsExport.ipynb pattern)
job_manager = JobStatusUpdater(db_client)

try:
    # Get job by request_id (matching productsExport.ipynb)
    job_record = await job_manager.get_job_by_request_id(job_id, saas_edge_id)
    
    if not job_record:
        raise ValueError(f"No job found with request_id={job_id} and saas_edge_id={saas_edge_id}")
    
    # Extract request_args (matching productsExport.ipynb)
    request_args = job_record.get("request_args", {})
    
    print("✅ Job details fetched successfully")
    print(f"Job Name: {job_record.get('job_name', 'N/A')}")
    print(f"Job Status: {job_record.get('job_status', 'N/A')}")
    print(f"\nRequest Args:")
    for key, value in request_args.items():
        print(f"  {key}: {value}")
    
    # Extract parameters from request_args (matching productsExport.ipynb structure)
    template_id = request_args.get("template_id")
    filter_params = request_args.get("filter_params", {})
    job_path = request_args.get("job_path", "")
    job_name = request_args.get("job_name", job_record.get("job_name", "Products Export"))
    is_xlsx = request_args.get("is_xlsx", False)
    is_json = request_args.get("is_json", False)
    email = request_args.get("email", "")
    export_with_readiness = request_args.get("export_with_readiness", False)
    
    # Determine file format
    if is_xlsx:
        file_format = "xlsx"
    elif is_json:
        file_format = "json"
    else:
        file_format = "csv"  # default
    
    print(f"\n✅ Extracted parameters:")
    print(f"  Template ID: {template_id}")
    print(f"  File Format: {file_format}")
    print(f"  Export with Readiness: {export_with_readiness}")
    
except Exception as e:
    print(f"❌ Failed to fetch job details: {e}")
    traceback.print_exc()
    raise


## 3. Generate Template Schema (if needed)


In [ ]:
# Generate or fetch template schema (matching productsExport.ipynb pattern)
if not template_id:
    raise ValueError("template_id is required for export")

# Generate or fetch template schema
try:
    template_schema = await get_or_generate_template_schema(db_client, saas_edge_id, template_id)
    print(f"✅ Template schema ready")
except Exception as e:
    print(f"⚠️  Schema generation failed: {e}")
    traceback.print_exc()


## 4. Add Utility Functions (Matching Products Notebooks)


In [ ]:
# Utility functions matching productsExport.ipynb pattern
GCP_BUCKET_NAME = os.getenv("GCP_BUCKET_NAME", "edge-assets")

def sanitize_path(path):
    """Sanitize path by replacing forward slashes with underscores"""
    return path.replace('/', '_').replace(".ipynb", "").strip()

def get_output_filename(saas_edge_id, job_path, file_type):
    """Generate standardized output filename with proper date formatting"""
    try:
        current_date = datetime.now()
        date_str = current_date.strftime('%Y-%m-%d')
        timestamp_str = current_date.strftime('%H%M%S')
        sanitized_job_path = sanitize_path(job_path)
        return f"{saas_edge_id}/catalog-edge/export-job-reports/{sanitized_job_path}/{date_str}/export-file-{timestamp_str}.{file_type}"
    except Exception as e:
        print(f"Error generating output filename: {str(e)}")
        return f"{saas_edge_id}/catalog-edge/job-reports/error_report.{file_type}"

def get_output_filename_failed(saas_edge_id, job_path, file_type):
    """Generate filename for failed export report"""
    try:
        current_date = datetime.now()
        date_str = current_date.strftime('%Y-%m-%d')
        timestamp_str = current_date.strftime('%H%M%S')
        sanitized_job_path = sanitize_path(job_path)
        return f"{saas_edge_id}/catalog-edge/export-job-reports/{sanitized_job_path}/{date_str}/export-file-failed{timestamp_str}.{file_type}"
    except Exception as e:
        print(f"Error generating output filename: {str(e)}")
        return f"{saas_edge_id}/catalog-edge/job-reports/failed_report.{file_type}"

async def upload_to_gcp_bucket(file_data, file_name, bucket_name, base_path=""):
    """Upload data to GCP bucket using Application Default Credentials"""
    try:
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        full_path = f"{base_path.rstrip('/')}/{file_name}" if base_path else file_name
        blob = bucket.blob(full_path)
        
        if isinstance(file_data, (list, dict)):
            upload_data = json.dumps(file_data, indent=2)
            blob.upload_from_string(upload_data, content_type='application/json')
        elif isinstance(file_data, str):
            blob.upload_from_string(file_data)
        elif isinstance(file_data, bytes):
            blob.upload_from_string(file_data, content_type='application/octet-stream')
        else:
            blob.upload_from_file(file_data)
        
        url = f"https://storage.googleapis.com/{bucket_name}/{full_path}"
        print(f"Successfully uploaded {file_name} to {url}")
        return True, url
    except Exception as e:
        print(f"Error uploading to GCP bucket: {str(e)}")
        traceback.print_exc()
        return False, ""

def save_to_xlsx(data_list, file_name="output.xlsx"):
    """Convert data list to XLSX file (matching productsExport.ipynb)"""
    success_count = 0
    failure_count = 0
    
    workbook = Workbook()
    sheet = workbook.active
    try:
        data_list = [item for item in data_list if isinstance(item, dict)]
        if data_list:
            first_item = data_list[0]
            headers = list(first_item.keys())
            sheet.append(headers)
            
            for data in data_list:
                try:
                    row = [data.get(header, '') for header in headers]
                    sheet.append(row)
                    success_count += 1
                except Exception as e:
                    print(f"ERROR: Failed to write row to XLSX: {str(e)}")
                    failure_count += 1
    except Exception as e:
        print_exc()
        print(f"ERROR: Failed to write row to XLSX: {str(e)}")
        failure_count += 1
    
    workbook.save(file_name)
    print(f"XLSX file saved as {file_name}")
    return file_name, success_count, failure_count

def save_to_json(data_list, file_name="output.json"):
    """Convert data list to JSON file (matching productsExport.ipynb)"""
    success_count = 0
    failure_count = 0
    
    json_data = []
    for data in data_list:
        try:
            json_data.append(data)
            success_count += 1
        except Exception as e:
            print(f"ERROR: Failed to process JSON data: {str(e)}")
            failure_count += 1
    
    with open(file_name, 'w') as json_file:
        json.dump(json_data, json_file, indent=2)
    print(f"JSON file saved as {file_name}")
    return file_name, success_count, failure_count

async def update_job_details(job_id, job_response, failed_url=None, success_url=None):
    """Update job details in database (matching productsExport.ipynb pattern)"""
    try:
        update_data = {
            "job_response": job_response,
            "updated_at": datetime.utcnow()
        }
        
        if failed_url:
            update_data["failed_job_summary_link"] = failed_url
        if success_url:
            update_data["success_job_summary_link"] = success_url
        
        await db_client.update(
            "saas_edge_jobs",
            {"request_id": job_id},
            update_data
        )
        
        print(f"✅ Job details updated")
        return True
    except Exception as e:
        print(f"❌ Failed to update job details: {e}")
        traceback.print_exc()
        return False

print("✅ Utility functions defined")


## 5. Fetch Products from Database


In [ ]:
# Fetch products from database based on filters (matching productsExport.ipynb pattern)
# Build query with filter_params (matching productsExport.ipynb export_products_to_dataframe)
import io

# Helper function to get sync DB connection (matching productsExport.ipynb pattern)
def get_db_connection_sync():
    """Get synchronous database connection for COPY operations"""
    import psycopg2
    import os
    
    # Get connection details from environment or config
    db_host = os.getenv("DB_HOST", "/cloudsql/saastify-base-wm:us-central1:saastify-pgdb-us")
    db_port = int(os.getenv("DB_PORT", "5432"))
    db_name = os.getenv("DB_NAME", "catalog-edge-db")
    db_user = os.getenv("DB_USER", "postgres")
    db_password = os.getenv("DB_PASSWORD", "")
    
    is_local = os.getenv("ENV", "product") == "local"
    
    if is_local:
        return psycopg2.connect(
            host=db_host,
            port=db_port,
            database=db_name,
            user=db_user,
            password=db_password
        )
    else:
        # Cloud SQL Unix socket
        return psycopg2.connect(
            host=db_host,
            port=db_port,
            database=db_name,
            user=db_user,
            password=db_password
        )

def export_products_to_dataframe(saas_edge_id, filter_params=None):
    """Export products to dataframe using COPY command (matching productsExport.ipynb)"""
    conn = None
    try:
        conn = get_db_connection_sync()
        csv_buffer = io.TextIOWrapper(io.BytesIO(), encoding='utf-8')
        
        # Start query with saas_edge_id filter (matching productsExport.ipynb)
        query_conditions = [f"saas_edge_id = '{saas_edge_id}'"]
        
        if filter_params:
            for key, condition in filter_params.items():
                if isinstance(condition, dict):
                    if 'in' in condition:
                        values = condition['in']
                        if key.lower() == 'productid' or key.lower() == 'product_id':
                            values_list = ','.join(str(int(v)) for v in values)
                            query_conditions.append(f"{key} IN ({values_list})")
                        else:
                            values_list = ','.join(f"'{v}'" for v in values)
                            query_conditions.append(f"{key} IN ({values_list})")
                    elif 'includesInsensitive' in condition:
                        value = condition['includesInsensitive']
                        query_conditions.append(f"LOWER({key}) LIKE '%{value.lower()}%'")
                    elif 'equalTo' in condition:
                        value = condition['equalTo']
                        if isinstance(value, (int, float)):
                            query_conditions.append(f"{key} = {value}")
                        else:
                            query_conditions.append(f"{key} = '{value}'")
                    elif 'notEqualTo' in condition:
                        value = condition['notEqualTo']
                        if isinstance(value, (int, float)):
                            query_conditions.append(f"{key} != {value}")
                        else:
                            query_conditions.append(f"{key} != '{value}'")
                    elif 'greaterThanOrEqualTo' in condition or 'lessThan' in condition:
                        gte = condition.get('greaterThanOrEqualTo')
                        lt = condition.get('lessThan')
                        if gte:
                            query_conditions.append(f"{key} >= '{gte}'")
                        if lt:
                            query_conditions.append(f"{key} < '{lt}'")
                else:
                    if isinstance(condition, (int, float)):
                        query_conditions.append(f"{key} = {condition}")
                    else:
                        query_conditions.append(f"{key} = '{condition}'")
        
        # Build final SQL query (using products table)
        final_conditions = " AND ".join(query_conditions)
        base_query = f"""
        COPY (
            SELECT * FROM products
            WHERE {final_conditions}
        ) TO STDOUT WITH CSV HEADER
        """
        
        with conn.cursor() as cur:
            cur.execute("SET statement_timeout = 0")
            cur.copy_expert(base_query, csv_buffer)
        
        # Read from start
        csv_buffer.seek(0)
        df = pd.read_csv(csv_buffer, keep_default_na=False, low_memory=False)
        return df
        
    finally:
        if conn:
            conn.close()

# Export products to dataframe
df = export_products_to_dataframe(saas_edge_id, filter_params)
products = df.to_dict(orient="records")

print(f"✅ Fetched {len(products)} products from database")
if products:
    print(f"\nSample product:")
    print(f"  ID: {products[0].get('product_id')}")
    print(f"  Name: {products[0].get('name')}")

# Update job details with total count (matching productsExport.ipynb)
total_count_data = len(products)
job_response = {
    "total": total_count_data,
    "success": 0,
    "failed": 0,
}
await update_job_details(job_id, job_response, failed_url=None, success_url=None)


## 6. Setup Logging


In [ ]:
# Setup structured logging (matching productsExport.ipynb)
setup_logging(level="INFO", structured=True)

# Set job context for logging
set_job_context(
    job_name=job_name,
    saas_edge_id=saas_edge_id,
    template_id=template_id
)

print("✅ Logging configured")


## 7. Run Export Pipeline (Using SDK)


In [ ]:
# Run export pipeline using SDK (matching productsExport.ipynb pattern)
# This follows the same orchestration pattern as productsExport.ipynb
# The SDK pipeline handles transformation, validation, and cache writing
# We then extract transformed data from completeness cache for file processing

print(f"🚀 Starting export job: {job_name}")
print(f"   Template: {template_id}")
print(f"   Products: {len(products)}")
print(f"   Format: {file_format}")
print(f"   Export with Readiness: {export_with_readiness}")
print()

start_time = datetime.now()

# Import export components
from saastify_edge.export.orchestrator import ExportPipeline, ExportPipelineConfig
from saastify_edge.db import CompletenessReader, CompletenessWriter
from saastify_edge.utils import get_or_generate_template_schema
from saastify_edge.import_pipeline.template_mapper import TemplateMapper
from saastify_edge.transformations.engine import apply_transformations
from saastify_edge.validation.engine import validate_row

# Create temporary local file path (not used since we process manually)
temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=f".{file_format}")
local_output_path = temp_file.name
temp_file.close()

# Configure file builder
file_config = {
    "delimiter": "," if file_format == "csv" else "\t" if file_format == "tsv" else None,
    "include_headers": True
}

# Initialize SDK components for transformation/validation
template_mapper = TemplateMapper(db_client=db_client)
completeness_reader = CompletenessReader(db_client=db_client)
completeness_writer = CompletenessWriter(db_client=db_client)

# Load template (matching productsExport.ipynb pattern)
template = await template_mapper.load_template(saas_edge_id, template_id)
if not template:
    raise ValueError(f"Template {template_id} not found")

print(f"✅ Template loaded: {template.get('template_name', template_id)}")

# Process each product (matching productsExport.ipynb orchestrate_workflow pattern)
result_list = []
failed_list = []

for product in products:
    product_id = product.get("product_id")
    if not product_id:
        continue
    
    try:
        recheck_resp = None
        
        # Check completeness cache if export_with_readiness is enabled (matching productsExport.ipynb)
        if export_with_readiness:
            completeness_record = await completeness_reader.get_record(
                saas_edge_id=saas_edge_id,
                template_id=template_id,
                product_id=str(product_id),
                check_freshness=True
            )
            
            if completeness_record and completeness_record.get("cache_freshness"):
                # Use cached transformed response
                result_list.append({
                    "transformed_response": completeness_record.get("transformed_response", {}),
                    "is_valid": completeness_record.get("is_valid", False),
                    "validation_errors": completeness_record.get("validation_errors", {}),
                    "product_id": product_id,
                    "template_id": template_id,
                    "saas_edge_id": saas_edge_id
                })
                continue
        
        # Transform and validate using SDK (matching SDK's import pipeline pattern)
        mapped_data = template_mapper.map_row_to_fields(
            raw_row=product,
            template=template
        )
        
        # Apply transformations (matching SDK's import pipeline pattern)
        transformed_data = {}
        for field_name, raw_value in mapped_data.items():
            transformations = template_mapper.get_transformation_pipeline(
                template=template,
                field_name=field_name
            )
            
            # Convert transformation steps to rule string (matching SDK pattern)
            if transformations:
                rule_parts = []
                for trans in transformations:
                    if trans.args:
                        # Format with args: "operation|arg1|arg2"
                        args_str = "|".join(str(v) for v in trans.args.values())
                        rule_parts.append(f"{trans.operation}|{args_str}")
                    else:
                        rule_parts.append(trans.operation)
                rule_string = " + ".join(rule_parts)
                transformed_value = apply_transformations(raw_value, rule_string)
            else:
                transformed_value = raw_value
            
            transformed_data[field_name] = transformed_value
        
        # Validate fields (matching SDK's import pipeline pattern)
        validation_errors = {}
        from saastify_edge.validation.engine import validate_field
        
        # Build field_validations dict for row-level validation
        field_validations = {}
        
        for field_name, value in transformed_data.items():
            validation_rules = template_mapper.get_validation_rules(
                template=template,
                field_name=field_name
            )
            
            if validation_rules:
                field_validations[field_name] = validation_rules
                field_errors = validate_field(
                    field_name=field_name,
                    value=value,
                    rules=validation_rules,
                    context=transformed_data
                )
                if field_errors:
                    validation_errors[field_name] = field_errors
        
        # Row-level validation (matching SDK pattern)
        from saastify_edge.validation.engine import validate_row
        is_valid_row, row_errors_dict, error_count = validate_row(transformed_data, field_validations)
        if row_errors_dict:
            validation_errors["_row"] = row_errors_dict
        
        is_valid = len(validation_errors) == 0
        
        validation_result = {
            "transformed_response": transformed_data,
            "is_valid": is_valid,
            "validation_errors": validation_errors,
            "product_id": str(product_id),
            "template_id": template_id,
            "saas_edge_id": saas_edge_id,
            "last_product_updated_at": product.get("updated_at"),
            "last_template_updated_at": template.get("updated_at")
        }
        
        # Write to completeness cache
        await completeness_writer.write_record(
            saas_edge_id=saas_edge_id,
            template_id=template_id,
            product_id=str(product_id),
            transformed_response=transformed_data,
            validation_errors=validation_errors,
            is_valid=is_valid,
            run_type="EXPORT"
        )
        
        result_list.append(validation_result)
        
    except Exception as e:
        print(f"Error processing product {product_id}: {e}")
        traceback.print_exc()
        failed_list.append({
            "product_id": product_id,
            "error": str(e)
        })

end_time = datetime.now()
duration = (end_time - start_time).total_seconds()

print(f"✅ Processing completed!")
print(f"  Processed: {len(result_list)}")
print(f"  Failed: {len(failed_list)}")
print(f"  Duration: {duration:.2f} seconds")

# Store results for next cell (matching productsExport.ipynb pattern)
output_list = result_list
failed_url_data = failed_list
total_count_data = len(products)


## 8. Process and Upload Files to GCS


In [ ]:
# Process data and upload to GCS (matching productsExport.ipynb process_data pattern)
# Extract transformed_response from result_list (matching productsExport.ipynb)

# Extract only the transformedResponse from the data list (matching productsExport.ipynb)
transformed_responses = [
    item.get('transformed_response', {}) if isinstance(item, dict) and 'transformed_response' in item else {}
    for item in output_list
]

print(f"Extracted {len(transformed_responses)} transformed responses")

success_count = 0
failure_count = 0
failed_url = None
success_url = None
json_url = None
xlsx_url = None

# Handle case where no transformed data but failed records exist (matching productsExport.ipynb)
if not transformed_responses and failed_url_data:
    print("No transformed data, but failed records exist.")
    try:
        df = pd.DataFrame(failed_url_data)
        csv_buffer = BytesIO()
        df.to_csv(csv_buffer, index=False)
        csv_buffer.seek(0)
        
        csv_filename = get_output_filename_failed(saas_edge_id, job_path, 'csv')
        success_csv, url_csv = await upload_to_gcp_bucket(csv_buffer, csv_filename, GCP_BUCKET_NAME)
        failed_url = url_csv if success_csv else None
        print(f"✅ Failed report uploaded: {failed_url}")
    except Exception as e:
        print(f"ERROR: Failed to upload failed report: {str(e)}")
        traceback.print_exc()
    
    job_response = {
        "total": total_count_data,
        "success": 0,
        "failed": len(failed_url_data),
    }
    await update_job_details(job_id, job_response, failed_url=failed_url, success_url=None)
else:
    # Process transformed responses (matching productsExport.ipynb)
    if transformed_responses:
        total_records = len(transformed_responses)
        
        if is_xlsx:
            try:
                xlsx_file, xlsx_success, xlsx_failure = save_to_xlsx(transformed_responses)
                if xlsx_file:
                    output_filename = get_output_filename(saas_edge_id, job_path, "xlsx")
                    success, xlsx_url = await upload_to_gcp_bucket(xlsx_file, output_filename, GCP_BUCKET_NAME)
                    if success:
                        success_url = xlsx_url
                    print(f"✅ XLSX uploaded: {xlsx_url}")
            except Exception as e:
                failure_count += 1
                traceback.print_exc()
                print(f"ERROR: Failed to process XLSX: {str(e)}")
        
        if is_json:
            try:
                json_file, json_success, json_failure = save_to_json(transformed_responses)
                if json_file:
                    output_filename = get_output_filename(saas_edge_id, job_path, "json")
                    success, json_url = await upload_to_gcp_bucket(json_file, output_filename, GCP_BUCKET_NAME)
                    if success:
                        success_url = json_url
                    print(f"✅ JSON uploaded: {json_url}")
            except Exception as e:
                failure_count += 1
                traceback.print_exc()
                print(f"ERROR: Failed to process JSON: {str(e)}")
        
        success_count += total_records
        
        # Handle failed items
        if failed_url_data:
            try:
                df = pd.DataFrame(failed_url_data)
                csv_buffer = BytesIO()
                df.to_csv(csv_buffer, index=False)
                csv_buffer.seek(0)
                
                csv_filename = get_output_filename_failed(saas_edge_id, job_path, 'csv')
                success_csv, url_csv = await upload_to_gcp_bucket(csv_buffer, csv_filename, GCP_BUCKET_NAME)
                failed_url = url_csv if success_csv else None
                print(f"✅ Failed report uploaded: {failed_url}")
            except Exception as e:
                print(f"ERROR: Failed to upload failed report: {str(e)}")
                traceback.print_exc()
        
        # Update job details (matching productsExport.ipynb)
        job_response = {
            "total": total_count_data,
            "success": success_count,
            "failed": failure_count,
        }
        
        if json_url:
            job_response["json_url"] = json_url
        if xlsx_url:
            job_response["xlsx_url"] = xlsx_url
        
        await update_job_details(job_id, job_response, failed_url=failed_url, success_url=success_url)
        
        print(f"\n✅ Export processing completed:")
        print(f"  Success: {success_count}")
        print(f"  Failed: {failure_count}")
        if success_url:
            print(f"  Success URL: {success_url}")
        if failed_url:
            print(f"  Failed URL: {failed_url}")


## 9. Final Summary


In [ ]:
# Generate final summary (matching productsExport.ipynb pattern)
summary = {
    "job_id": job_id,
    "job_name": job_name,
    "status": "completed",
    "template_id": template_id,
    "total_rows": total_count_data,
    "success_rows": success_count,
    "failed_rows": failure_count,
    "file_format": file_format,
    "json_url": json_url,
    "xlsx_url": xlsx_url,
    "failed_url": failed_url,
    "success_url": success_url,
    "duration_seconds": duration,
    "completed_at": datetime.utcnow().isoformat()
}

print("\n" + "="*60)
print("PRODUCTS EXPORT JOB SUMMARY")
print("="*60)
for key, value in summary.items():
    print(f"{key}: {value}")

# Store summary as JSON for papermill output
summary_json = json.dumps(summary, indent=2)
print("\nSummary JSON:")
print(summary_json)


## 10. Cleanup


In [ ]:
# Disconnect from database
await db_client.disconnect()
print("✅ Database connection closed")
